# 5. Regression Model

### Cell 1 — Overview, imports, global config

In [1]:
# ================================================================
# Cell 1: Imports & global configuration
# ================================================================

import warnings
warnings.filterwarnings("ignore")

import os
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Optional XGBoost; if not available, fall back to GradientBoosting
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False

print("=" * 70)
print("REGRESSION MODEL EVALUATION – FINAL EXPERIMENT PIPELINE")
print("=" * 70)

# ---------------- Paths & configuration ----------------
PROJECT_ROOT = Path("..")

PREP_DIR = PROJECT_ROOT / "output_Preprocessing_TemporalDataSplitting"
FE_DIR   = PROJECT_ROOT / "output_FeatureEngineering"

FE_TRAIN_ORIG_DIR  = FE_DIR / "train" / "orig"
FE_TRAIN_CLEAN_DIR = FE_DIR / "train" / "cleaned"
FE_TEST_DIR        = FE_DIR / "test"

RESULTS_DIR  = PROJECT_ROOT / "results_reg"
FIG_DIR      = RESULTS_DIR / "figs"
ANALYSIS_DIR = RESULTS_DIR / "analysis"

RESULTS_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)
ANALYSIS_DIR.mkdir(parents=True, exist_ok=True)

# Horizons and feature types follow the classification setup
HORIZONS   = [1, 6, 12, 24]
FE_TYPES   = ["hourly", "daily", "merge"]
DATA_VERS  = ["orig", "cleaned"]

# We focus on four pollutants (NMHC was dropped in preprocessing)
POLLUTANTS = ["CO(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]

RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")

print("\nExperiment configuration:")
print(f"Horizons: {HORIZONS}")
print(f"FE types: {FE_TYPES}")
print(f"Data versions: {DATA_VERS}")
print(f"Results dir: {RESULTS_DIR}")
print(f"Run tag: {RUN_TAG}")

# ---------------- Load base data for targets ----------------
print("\n[STEP 0] Loading base data for target construction...")

base_df = pd.read_csv(
    PREP_DIR / "preprocessed_data.csv",
    index_col="DateTime",
    parse_dates=True,
)
base_df = base_df.sort_index()

missing_pollutants = [p for p in POLLUTANTS if p not in base_df.columns]
if missing_pollutants:
    raise KeyError(f"Missing pollutant columns in base_df: {missing_pollutants}")

print(f"base_df shape: {base_df.shape}")
print(f"base_df range: {base_df.index.min()} → {base_df.index.max()}")
print(f"Years present: {sorted(base_df.index.year.unique())}")

REGRESSION MODEL EVALUATION – FINAL EXPERIMENT PIPELINE

Experiment configuration:
Horizons: [1, 6, 12, 24]
FE types: ['hourly', 'daily', 'merge']
Data versions: ['orig', 'cleaned']
Results dir: ../results_reg
Run tag: 20251119_020758

[STEP 0] Loading base data for target construction...
base_df shape: (8833, 12)
base_df range: 2004-03-10 18:00:00 → 2005-04-04 14:00:00
Years present: [2004, 2005]


### Cell 2 — Helpers: loading data, labels, models, confusion matrix

In [2]:
# ================================================================
# Cell 2: Helper functions (targets, loaders, models, metrics, plots)
# ================================================================

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ---------------- Target construction ----------------
def make_future_reg_target(df: pd.DataFrame, pollutant: str, h: int) -> pd.Series:
    """
    Build y_{t+h} for a given pollutant and horizon h.
    """
    if pollutant not in df.columns:
        raise KeyError(f"{pollutant} not found in base_df")
    y_future = df[pollutant].shift(-h)
    y_future = y_future.dropna()
    return y_future


# ---------------- Feature loaders ----------------
def load_train_test_fe(fe_type: str, data_ver: str):
    """
    Load 2004 train and 2005 test feature tables for a given
    feature type (hourly / daily / merge) and data version.
    """
    if fe_type not in FE_TYPES:
        raise ValueError(f"Unknown FE type: {fe_type}")

    if data_ver == "orig":
        train_path = FE_TRAIN_ORIG_DIR / f"train_2004_fe_{fe_type}_orig.csv"
    elif data_ver == "cleaned":
        train_path = FE_TRAIN_CLEAN_DIR / f"train_2004_fe_{fe_type}_cleaned.csv"
    else:
        raise ValueError(f"Unknown data version: {data_ver}")

    test_path = FE_TEST_DIR / f"test_2005_fe_{fe_type}.csv"

    if not train_path.exists():
        raise FileNotFoundError(f"Train FE file not found: {train_path}")
    if not test_path.exists():
        raise FileNotFoundError(f"Test FE file not found: {test_path}")

    X_tr = pd.read_csv(train_path, index_col="DateTime", parse_dates=True)
    X_te = pd.read_csv(test_path, index_col="DateTime", parse_dates=True)

    X_tr = X_tr.sort_index()
    X_te = X_te.sort_index()

    return X_tr, X_te


def build_train_test_for_fe(fe_type: str, data_ver: str, pollutant: str, h: int):
    """
    For a given FE type + data version + pollutant + horizon,
    build aligned (X_train, y_train, X_test, y_test) with
    time-respecting splits.
    """
    X_tr, X_te = load_train_test_fe(fe_type, data_ver)
    y_full = make_future_reg_target(base_df, pollutant, h)

    # Train: 2004 only, and avoid labels that would use 2005
    idx_tr = X_tr.index.intersection(y_full.index)
    boundary = pd.Timestamp("2004-12-31 23:00:00") - pd.Timedelta(hours=h - 1)
    idx_tr = idx_tr[idx_tr <= boundary]
    X_train = X_tr.loc[idx_tr]
    y_train = y_full.loc[idx_tr]

    # Test: 2005 only
    idx_te = X_te.index.intersection(y_full.index)
    X_test = X_te.loc[idx_te]
    y_test = y_full.loc[idx_te]

    return X_train, y_train, X_test, y_test


# ---------------- Model zoo ----------------
def model_zoo_reg():
    """
    Regression models:
      - Linear Regression
      - Random Forest Regressor
      - XGBoost Regressor (or GradientBoostingRegressor as fallback)
    """
    models = {}

    models["LinReg"] = LinearRegression()

    models["RF"] = RandomForestRegressor(
        n_estimators=400,
        random_state=42,
        n_jobs=-1,
    )

    if HAS_XGB:
        models["XGB"] = XGBRegressor(
            n_estimators=400,
            max_depth=5,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            objective="reg:squarederror",
            random_state=42,
            n_jobs=-1,
        )
    else:
        models["GB"] = GradientBoostingRegressor(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=3,
            random_state=42,
        )

    return models


# ---------------- Metrics ----------------
def eval_reg_metrics(y_true, y_pred) -> dict:
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    return {"RMSE": rmse, "MAE": mae, "R2": r2}


# ---------------- Simple residual / time-series plots ----------------
def save_ts_residual_plots(
    y_true: pd.Series,
    y_pred: np.ndarray,
    title_prefix: str,
    out_dir: Path,
    sample_n: int = 500,
):
    """
    Save two small figures:
      1) observed vs predicted over time (first sample_n points)
      2) residuals over time (same window)
    """
    if len(y_true) == 0:
        return

    out_dir.mkdir(parents=True, exist_ok=True)
    y_true = y_true.sort_index()
    y_pred_series = pd.Series(y_pred, index=y_true.index)
    residuals = y_true - y_pred_series

    sample_true = y_true.iloc[:sample_n]
    sample_pred = y_pred_series.iloc[:sample_n]
    sample_res  = residuals.iloc[:sample_n]

    # Observed vs predicted
    plt.figure(figsize=(7, 3))
    plt.plot(sample_true.index, sample_true.values, label="Observed", linewidth=1.0)
    plt.plot(sample_pred.index, sample_pred.values, label="Predicted", linewidth=1.0)
    plt.xlabel("Time")
    plt.ylabel("Concentration")
    plt.title(f"{title_prefix} – observed vs predicted")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_dir / f"{title_prefix}_ts.png", dpi=220)
    plt.close()

    # Residuals
    plt.figure(figsize=(7, 3))
    plt.plot(sample_res.index, sample_res.values, linewidth=1.0)
    plt.axhline(0.0, color="black", linewidth=0.8)
    plt.xlabel("Time")
    plt.ylabel("Residual")
    plt.title(f"{title_prefix} – residuals")
    plt.tight_layout()
    plt.savefig(out_dir / f"{title_prefix}_residuals.png", dpi=220)
    plt.close()

### Cell 3 — Run full experiment grid, save summary_full_run_*.csv + confusion matrices

In [3]:
# ================================================================
# Cell 3: Run full experiment grid
# ================================================================

results = []

print("\n[STEP 1] Running regression experiment grid...")

# ---------------- Naive baseline ----------------
def eval_naive_persistence(pollutant: str, h: int):
    """
    Persistence baseline:
      y_{t+h} = value of pollutant at t+h
      y_hat   = value of pollutant at t
    Evaluated on 2005 only.
    """
    if pollutant not in base_df.columns:
        raise KeyError(f"{pollutant} not in base_df")

    y_true_full = make_future_reg_target(base_df, pollutant, h)

    y_t = base_df[pollutant]
    idx = y_true_full.index.intersection(y_t.index)

    y_true = y_true_full.loc[idx]
    y_hat  = y_t.loc[idx]

    # Test: 2005 only
    test_idx = idx[idx.year == 2005]
    y_te = y_true.loc[test_idx]
    yhat_te = y_hat.loc[test_idx]

    metrics = eval_reg_metrics(y_te, yhat_te)
    return metrics, y_te, yhat_te


# 1) Naive baseline per pollutant × horizon
for pollutant in POLLUTANTS:
    for h in HORIZONS:
        print(f"\n[Naive] pollutant={pollutant}, h={h}")
        naive_metrics, y_te_naive, yhat_naive = eval_naive_persistence(pollutant, h)

        results.append({
            "pollutant": pollutant,
            "h": h,
            "fe_type": "N/A",
            "data_ver": "N/A",
            "model": "Naive",
            "train_n": np.nan,
            "train_n_eff": np.nan,
            "clean_removed": np.nan,
            **naive_metrics,
        })

        if pollutant == "CO(GT)" and h in (1, 24):
            title_prefix = f"naive_{pollutant.replace('(GT)','')}_h{h}"
            save_ts_residual_plots(
                y_te_naive,
                yhat_naive.values,
                title_prefix=title_prefix,
                out_dir=FIG_DIR,
            )


model_dict = model_zoo_reg()

for pollutant in POLLUTANTS:
    for h in HORIZONS:
        print("\n" + "=" * 60)
        print(f"[GRID] pollutant={pollutant}  h={h}")
        print("=" * 60)

        for fe_type in FE_TYPES:
            for data_ver in DATA_VERS:
                print(f"FE={fe_type:6s} | data={data_ver:7s}")

                X_tr, y_tr, X_te, y_te = build_train_test_for_fe(
                    fe_type, data_ver, pollutant, h
                )
                n_train = len(X_tr)

                if n_train == 0 or len(X_te) == 0:
                    print("  Empty train or test set, skipping.")
                    continue

                if data_ver == "orig":
                    clean_removed = 0
                else:
                    X_tr_orig, _, _, _ = build_train_test_for_fe(
                        fe_type, "orig", pollutant, h
                    )
                    clean_removed = max(0, len(X_tr_orig) - len(X_tr))

                print(
                    f"Train size={len(X_tr)}, Test size={len(X_te)}, "
                    f"Removed by cleaning={clean_removed}"
                )

                scaler = StandardScaler().fit(X_tr)
                X_tr_s = scaler.transform(X_tr)
                X_te_s = scaler.transform(X_te)

                for m_name, proto in model_dict.items():
                    print(f"  Model: {m_name}")

                    model_cls = proto.__class__
                    model = model_cls(**proto.get_params())

                    model.fit(X_tr_s, y_tr)
                    yhat = model.predict(X_te_s)
                    metrics = eval_reg_metrics(y_te, yhat)

                    results.append({
                        "pollutant": pollutant,
                        "h": h,
                        "fe_type": fe_type,
                        "data_ver": data_ver,
                        "model": m_name,
                        "train_n": int(n_train),
                        "train_n_eff": int(len(X_tr)),
                        "clean_removed": int(clean_removed),
                        **metrics,
                    })

                    if (
                        pollutant == "CO(GT)"
                        and fe_type == "merge"
                        and data_ver == "cleaned"
                        and m_name in ("RF", "XGB", "GB", "LinReg")
                        and h in (1, 24)
                    ):
                        safe_name = m_name.lower()
                        title_prefix = f"{safe_name}_CO_merge_cleaned_h{h}"
                        save_ts_residual_plots(
                            y_te,
                            yhat,
                            title_prefix=title_prefix,
                            out_dir=FIG_DIR,
                        )

# 3) Save summary table
df_res_reg = pd.DataFrame(results)
df_res_reg = df_res_reg[
    [
        "pollutant",
        "h",
        "fe_type",
        "data_ver",
        "model",
        "train_n",
        "train_n_eff",
        "clean_removed",
        "RMSE",
        "MAE",
        "R2",
    ]
].sort_values(["pollutant", "h", "model", "fe_type", "data_ver"])

summary_path_reg = RESULTS_DIR / f"summary_full_run_{RUN_TAG}.csv"
df_res_reg.to_csv(summary_path_reg, index=False)

print("\nRegression grid finished.")
print(f"Summary saved to: {summary_path_reg}")
df_res_reg.head()


[STEP 1] Running regression experiment grid...

[Naive] pollutant=CO(GT), h=1

[Naive] pollutant=CO(GT), h=6

[Naive] pollutant=CO(GT), h=12

[Naive] pollutant=CO(GT), h=24

[Naive] pollutant=C6H6(GT), h=1

[Naive] pollutant=C6H6(GT), h=6

[Naive] pollutant=C6H6(GT), h=12

[Naive] pollutant=C6H6(GT), h=24

[Naive] pollutant=NOx(GT), h=1

[Naive] pollutant=NOx(GT), h=6

[Naive] pollutant=NOx(GT), h=12

[Naive] pollutant=NOx(GT), h=24

[Naive] pollutant=NO2(GT), h=1

[Naive] pollutant=NO2(GT), h=6

[Naive] pollutant=NO2(GT), h=12

[Naive] pollutant=NO2(GT), h=24

[GRID] pollutant=CO(GT)  h=1
FE=hourly | data=orig   
Train size=6590, Test size=2230, Removed by cleaning=0
  Model: LinReg
  Model: RF
  Model: XGB
FE=hourly | data=cleaned
Train size=6414, Test size=2230, Removed by cleaning=176
  Model: LinReg
  Model: RF
  Model: XGB
FE=daily  | data=orig   
Train size=252, Test size=93, Removed by cleaning=0
  Model: LinReg
  Model: RF
  Model: XGB
FE=daily  | data=cleaned
Train size=252,

,pollutant,h,fe_type,data_ver,model,train_n,train_n_eff,clean_removed,RMSE,MAE,R2
97,C6H6(GT),1,daily,cleaned,LinReg,252.0,252.0,0.0,2.545233,2.161793,0.366256
94,C6H6(GT),1,daily,orig,LinReg,252.0,252.0,0.0,2.412601,2.038110,0.430584
91,C6H6(GT),1,hourly,cleaned,LinReg,6414.0,6414.0,176.0,3.495601,2.532470,0.708231
88,C6H6(GT),1,hourly,orig,LinReg,6590.0,6590.0,0.0,3.832434,2.914623,0.649293
103,C6H6(GT),1,merge,cleaned,LinReg,5845.0,5845.0,161.0,3.050386,2.148845,0.777820


### Cell 4 — Analysis by four dimensions (RQ1–RQ4) + plots + CSVs

In [4]:
# ================================================================
# Cell 4: Post-hoc analysis for RQ1–RQ4 (regression)
# ================================================================

print("\n[STEP 2] Post-hoc analysis for regression experiments...")

df_all   = df_res_reg.copy()
df_naive = df_all[df_all["model"] == "Naive"].copy()
df_models = df_all[df_all["model"] != "Naive"].copy()

PRIMARY = "RMSE"


# ------------------------------------------------------------
# RQ1: Model family comparison (cleaned, FE=merge)
#      averaged over pollutants
# ------------------------------------------------------------
print("\n[RQ1] Model comparison (cleaned data, FE=merge)...")

rq1 = df_models[
    (df_models["data_ver"] == "cleaned")
    & (df_models["fe_type"] == "merge")
].copy()

rq1_summary = (
    rq1.groupby(["h", "model"])[["RMSE", "MAE", "R2"]]
       .mean()
       .reset_index()
)

rq1_path = ANALYSIS_DIR / "rq1_model_comparison_reg.csv"
rq1_summary.to_csv(rq1_path, index=False)
print(f"Saved RQ1 summary to: {rq1_path}")

plt.figure(figsize=(6, 4))
for m in sorted(rq1["model"].unique()):
    sub = rq1_summary[rq1_summary["model"] == m]
    plt.plot(sub["h"], sub[PRIMARY], marker="o", label=m)

plt.xlabel("Horizon h (hours)")
plt.ylabel(PRIMARY)
plt.title(f"RQ1 – Model comparison (cleaned, FE=merge, metric={PRIMARY})")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIG_DIR / "rq1_model_comparison_reg.png", dpi=220)
plt.close()


# ------------------------------------------------------------
# RQ2: Effect of anomaly cleaning (orig vs cleaned)
#       ΔRMSE = cleaned - orig
# ------------------------------------------------------------
print("\n[RQ2] Effect of anomaly cleaning (orig vs cleaned)...")

def compute_clean_delta_reg(group):
    if set(group["data_ver"]) >= {"orig", "cleaned"}:
        m_clean = group[group["data_ver"] == "cleaned"][["RMSE", "MAE", "R2"]].mean()
        m_orig  = group[group["data_ver"] == "orig"][["RMSE", "MAE", "R2"]].mean()
        return (m_clean - m_orig)
    else:
        return pd.Series({"RMSE": np.nan, "MAE": np.nan, "R2": np.nan})

rq2_delta = (
    df_models.groupby(["pollutant", "h", "fe_type", "model"])
             .apply(compute_clean_delta_reg)
             .reset_index()
)

rq2_path = ANALYSIS_DIR / "rq2_anomaly_effect_reg.csv"
rq2_delta.to_csv(rq2_path, index=False)
print(f"Saved RQ2 delta table to: {rq2_path}")

rq2_fe_summary = (
    rq2_delta.groupby("fe_type")[["RMSE", "MAE", "R2"]]
             .mean()
             .reset_index()
)

plt.figure(figsize=(5, 3.5))
plt.bar(rq2_fe_summary["fe_type"], rq2_fe_summary["RMSE"])
plt.xlabel("FE type")
plt.ylabel("Δ RMSE (cleaned - orig)")
plt.title("RQ2 – Average cleaning effect by FE type")
plt.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.savefig(FIG_DIR / "rq2_cleaning_effect_reg_by_fe.png", dpi=220)
plt.close()


# ------------------------------------------------------------
# RQ3: Feature engineering effect (cleaned only)
#      average over models and pollutants
# ------------------------------------------------------------
print("\n[RQ3] Feature engineering effect (cleaned data only)...")

rq3 = df_models[df_models["data_ver"] == "cleaned"].copy()

rq3_summary = (
    rq3.groupby(["h", "fe_type"])[["RMSE", "MAE", "R2"]]
       .mean()
       .reset_index()
)

rq3_path = ANALYSIS_DIR / "rq3_fe_effect_reg.csv"
rq3_summary.to_csv(rq3_path, index=False)
print(f"Saved RQ3 summary to: {rq3_path}")

pivot = rq3_summary.pivot(index="fe_type", columns="h", values=PRIMARY)

plt.figure(figsize=(6, 3.5))
im = plt.imshow(pivot.values, aspect="auto", cmap="viridis")
plt.colorbar(im, fraction=0.046, pad=0.04, label=PRIMARY)
plt.xticks(range(len(pivot.columns)), pivot.columns)
plt.yticks(range(len(pivot.index)), pivot.index)
plt.xlabel("Horizon h (hours)")
plt.ylabel("FE type")
plt.title(f"RQ3 – Feature engineering effect ({PRIMARY})")

for i in range(pivot.shape[0]):
    for j in range(pivot.shape[1]):
        val = pivot.values[i, j]
        plt.text(
            j,
            i,
            f"{val:.2f}",
            ha="center",
            va="center",
            color="white" if val < pivot.values.max() / 2 else "black",
            fontsize=8,
        )

plt.tight_layout()
plt.savefig(FIG_DIR / "rq3_fe_effect_reg_heatmap.png", dpi=220)
plt.close()


# ------------------------------------------------------------
# RQ4: Horizon effect (average over models / FE / versions / pollutants)
# ------------------------------------------------------------
print("\n[RQ4] Horizon effect (average over models, FE, data versions, pollutants)...")

rq4_summary = (
    df_models.groupby("h")[["RMSE", "MAE", "R2"]]
             .mean()
             .reset_index()
)

rq4_path = ANALYSIS_DIR / "rq4_horizon_effect_reg.csv"
rq4_summary.to_csv(rq4_path, index=False)
print(f"Saved RQ4 summary to: {rq4_path}")

plt.figure(figsize=(6, 4))
plt.plot(rq4_summary["h"], rq4_summary["RMSE"], marker="o", label="RMSE")
plt.plot(rq4_summary["h"], rq4_summary["MAE"], marker="o", label="MAE")
plt.xlabel("Horizon h (hours)")
plt.ylabel("Error")
plt.title("RQ4 – Average regression error vs horizon")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIG_DIR / "rq4_reg_error_vs_horizon.png", dpi=220)
plt.close()

print("\nRegression analysis complete.")
print(f"Full grid results: {summary_path_reg}")
print(f"Analysis CSVs   : {ANALYSIS_DIR}")
print(f"Figures         : {FIG_DIR}")


[STEP 2] Post-hoc analysis for regression experiments...

[RQ1] Model comparison (cleaned data, FE=merge)...
Saved RQ1 summary to: ../results_reg/analysis/rq1_model_comparison_reg.csv

[RQ2] Effect of anomaly cleaning (orig vs cleaned)...
Saved RQ2 delta table to: ../results_reg/analysis/rq2_anomaly_effect_reg.csv

[RQ3] Feature engineering effect (cleaned data only)...
Saved RQ3 summary to: ../results_reg/analysis/rq3_fe_effect_reg.csv

[RQ4] Horizon effect (average over models, FE, data versions, pollutants)...
Saved RQ4 summary to: ../results_reg/analysis/rq4_horizon_effect_reg.csv

Regression analysis complete.
Full grid results: ../results_reg/summary_full_run_20251119_020758.csv
Analysis CSVs   : ../results_reg/analysis
Figures         : ../results_reg/figs
